# YOLOv12 Nano COCO Benchmark

### Setup Dependencies

In [1]:
import os
import json
import time
import torch
import numpy as np
import psutil
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.torch as fout
from fiftyone import ViewField as F

from ultralytics import YOLO

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA/GPU available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU Only'}")
print(f"CPU Cores: {psutil.cpu_count(logical=True)}")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\amanp\AppData\Roaming\yolov12\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
FlashAttention is not available on this device. Using scaled_dot_product_attention instead.
PyTorch version: 2.5.1+cu121
CUDA/GPU available: True
CUDA version: 12.1
GPU name: NVIDIA GeForce GTX 1660 Ti
CPU Cores: 12


### Configure Local COCO Paths

In [5]:
COCO_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "coco-dataset", "coco-2017"))

VAL_IMAGES = os.path.join(COCO_ROOT, "validation", "data")
VAL_ANN = os.path.join(COCO_ROOT, "validation", "labels.json")

print("Images path:", VAL_IMAGES)
print("Annotation file:", VAL_ANN)
print("Images found:", len(os.listdir(VAL_IMAGES)))

Images path: C:\object-identification-uav-camera\phase-1\coco-dataset\coco-2017\validation\data
Annotation file: C:\object-identification-uav-camera\phase-1\coco-dataset\coco-2017\validation\labels.json
Images found: 3125


### Load COCO Dataset in FiftyOne

In [6]:
dataset = fo.load_dataset("coco-val-local")
session = fo.launch_app(dataset)

### Initialize Model

In [7]:
print("Loading YOLOv12 Nano...")
model_yolo12 = YOLO("yolov12n.pt")   # Nano version
print("YOLOv12 Nano initialized.")

Loading YOLOv12 Nano...
YOLOv12 Nano initialized.


### Jetson Simulation Setup Cell

In [9]:
# ============================
# Jetson Orin Nano 8GB Simulation Mode
# ============================

print("\nEnabling Jetson Orin Nano 8GB simulation constraints...\n")

# ---- 1. Disable cuDNN fast paths (Jetson uses slower kernels) ----
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False
torch.set_float32_matmul_precision("high")
print("Disabled cuDNN fast algorithms & TF32.")

# ---- 2. Limit CPU parallelism ----
import os
torch.set_num_threads(2)
os.environ["OMP_NUM_THREADS"] = "2"
print("CPU threads limited to 2 (Jetson-like).")

# ---- 3. I/O latency to mimic Jetson eMMC/SD card ----
IO_LATENCY = 0.004  # 4 ms
print("Added 4ms I/O latency per image.")

# ---- 4. Simulate Jetson memory bandwidth ----
BANDWIDTH_LATENCY = 0.0008  # 0.8 ms
print("Added memory bandwidth stall of 0.8ms.")

# ---- 5. Simulated GPU compute stall ----
# Jetson Orin Nano runs ~2–4× slower than GTX 1660 Ti for transformer models
GPU_STALL = 0.003  # 3 ms (tuneable)
print("Added 3ms GPU stall per inference (Jetson compute speed).")

# ---- 6. Constrain VRAM usage ----
torch.cuda.set_per_process_memory_fraction(0.80)
print("GPU VRAM restricted to ~80% of your card (Jetson-equivalent usable memory).")

print("\nJetson Simulation Mode READY.\n")


Enabling Jetson Orin Nano 8GB simulation constraints...

Disabled cuDNN fast algorithms & TF32.
CPU threads limited to 2 (Jetson-like).
Added 4ms I/O latency per image.
Added memory bandwidth stall of 0.8ms.
Added 3ms GPU stall per inference (Jetson compute speed).
GPU VRAM restricted to ~80% of your card (Jetson-equivalent usable memory).

Jetson Simulation Mode READY.



### Need to Map COCO Classes to IDS

In [14]:
# Build COCO name → COCO ID mapping (for the 9-class subset)
coco_name_to_id = {cat["name"]: cat["id"] for cat in coco_data["categories"]}

# YOLO name mapping: index → class name
yolo_names = model_yolo12.names  # e.g. {0:'person', 1:'bicycle', ...}

# Build YOLO → COCO category ID mapping but ONLY for your 9 classes
yolo_to_coco = {}

for yolo_idx, class_name in yolo_names.items():
    if class_name in coco_name_to_id:   # keep only the 9 classes
        yolo_to_coco[yolo_idx] = coco_name_to_id[class_name]

print(yolo_to_coco)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 13, 12: 14, 13: 15, 14: 16, 15: 17, 16: 18, 17: 19, 18: 20, 19: 21, 20: 22, 21: 23, 22: 24, 23: 25, 24: 27, 25: 28, 26: 31, 27: 32, 28: 33, 29: 34, 30: 35, 31: 36, 32: 37, 33: 38, 34: 39, 35: 40, 36: 41, 37: 42, 38: 43, 39: 44, 40: 46, 41: 47, 42: 48, 43: 49, 44: 50, 45: 51, 46: 52, 47: 53, 48: 54, 49: 55, 50: 56, 51: 57, 52: 58, 53: 59, 54: 60, 55: 61, 56: 62, 57: 63, 58: 64, 59: 65, 60: 67, 61: 70, 62: 72, 63: 73, 64: 74, 65: 75, 66: 76, 67: 77, 68: 78, 69: 79, 70: 80, 71: 81, 72: 82, 73: 84, 74: 85, 75: 86, 76: 87, 77: 88, 78: 89, 79: 90}


### Inference Loop with Jestson Nano 8GB Conditions

In [15]:
print("Running Jetson-simulated YOLOv12 Nano inference...")

preds_json_yolo12_jetson = []
image_filenames = os.listdir(VAL_IMAGES)

with open(VAL_ANN, 'r') as f:
    coco_data = json.load(f)

# Build mappings
coco_name_to_id = {cat["name"]: cat["id"] for cat in coco_data["categories"]}
yolo_names = model_yolo12.names

# YOLO index → COCO category ID (only 9 classes)
yolo_to_coco = {
    yolo_idx: coco_name_to_id[class_name]
    for yolo_idx, class_name in yolo_names.items()
    if class_name in coco_name_to_id
}

print("YOLO → COCO mapping used:", yolo_to_coco)

image_id_map = {img["file_name"]: img["id"] for img in coco_data["images"]}

for filename in tqdm(image_filenames):
    time.sleep(IO_LATENCY)

    img_path = os.path.join(VAL_IMAGES, filename)

    # Load original resolution (do NOT resize!)
    with Image.open(img_path) as im:
        if im.mode != "RGB":
            im = im.convert("RGB")
        temp_path = "temp_y12.jpg"
        im.save(temp_path)

    time.sleep(BANDWIDTH_LATENCY)
    time.sleep(GPU_STALL)

    results = model_yolo12.predict(temp_path, conf=0.001, verbose=False)
    det = results[0]
    image_id = image_id_map[filename]

    if det.boxes is not None:
        for b in det.boxes:
            cls_id = int(b.cls)

            # Skip detections NOT in our 9-class subset
            if cls_id not in yolo_to_coco:
                continue

            coco_id = yolo_to_coco[cls_id]

            x1, y1, x2, y2 = b.xyxy.cpu().numpy()[0]
            w = x2 - x1
            h = y2 - y1

            preds_json_yolo12_jetson.append({
                "image_id": image_id,
                "category_id": coco_id,    # FIXED
                "score": float(b.conf),
                "bbox": [float(x1), float(y1), float(w), float(h)]
            })

with open("predictions_yolo12_jetson.json", "w") as f:
    json.dump(preds_json_yolo12_jetson, f)

print("Jetson-simulated YOLOv12 Nano inference complete.")

Running Jetson-simulated YOLOv12 Nano inference...
YOLO → COCO mapping used: {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 13, 12: 14, 13: 15, 14: 16, 15: 17, 16: 18, 17: 19, 18: 20, 19: 21, 20: 22, 21: 23, 22: 24, 23: 25, 24: 27, 25: 28, 26: 31, 27: 32, 28: 33, 29: 34, 30: 35, 31: 36, 32: 37, 33: 38, 34: 39, 35: 40, 36: 41, 37: 42, 38: 43, 39: 44, 40: 46, 41: 47, 42: 48, 43: 49, 44: 50, 45: 51, 46: 52, 47: 53, 48: 54, 49: 55, 50: 56, 51: 57, 52: 58, 53: 59, 54: 60, 55: 61, 56: 62, 57: 63, 58: 64, 59: 65, 60: 67, 61: 70, 62: 72, 63: 73, 64: 74, 65: 75, 66: 76, 67: 77, 68: 78, 69: 79, 70: 80, 71: 81, 72: 82, 73: 84, 74: 85, 75: 86, 76: 87, 77: 88, 78: 89, 79: 90}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [05:37<00:00,  9.25it/s]


Jetson-simulated YOLOv12 Nano inference complete.


### COCO Evaluation

In [16]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

print("Running COCO evaluation for Jetson-simulated YOLOv12 Nano...")

coco_gt = COCO(VAL_ANN)
coco_dt_yolo12_jetson = coco_gt.loadRes("predictions_yolo12_jetson.json")

coco_eval_yolo12_jetson = COCOeval(coco_gt, coco_dt_yolo12_jetson, "bbox")
coco_eval_yolo12_jetson.evaluate()
coco_eval_yolo12_jetson.accumulate()
coco_eval_yolo12_jetson.summarize()

Running COCO evaluation for Jetson-simulated YOLOv12 Nano...
loading annotations into memory...
Done (t=1.27s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.77s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=30.41s).
Accumulating evaluation results...
DONE (t=4.59s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.323
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.469
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.348
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.163
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.394
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.562
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.438
 Av

In [17]:
print("### YOLOv12 Nano (Jetson-Simulated) Accuracy Metrics\n")

# ---------------------------
# 1. mAP metrics
# ---------------------------
mAP_50_95 = coco_eval_yolo12_jetson.stats[0]     # AP@[0.50:0.95]
mAP_50    = coco_eval_yolo12_jetson.stats[1]     # AP@0.50

print(f"mAP@50:95: {mAP_50_95:.3f}")
print(f"mAP@50:    {mAP_50:.3f}\n")

# ---------------------------
# 2. Precision / Recall from COCOeval
# ---------------------------
precision = coco_eval_yolo12_jetson.eval['precision']   # precision[IoU, recall, class, area, maxDets]

# IoU index 0 = IoU=0.50, area=all, maxDets=100 (index=2)
prec_valid = precision[0, :, :, 0, 2]
prec_valid = prec_valid[prec_valid > -1]
prec_mean = np.mean(prec_valid)

recall = coco_eval_yolo12_jetson.eval['recall']         # recall[IoU, class, area, maxDets]
rec_valid = recall[0, :, 0, 2]
rec_valid = rec_valid[rec_valid > -1]
rec_mean = np.mean(rec_valid)

# ---------------------------
# 3. F1-score
# ---------------------------
if prec_mean + rec_mean == 0:
    f1 = 0
else:
    f1 = 2 * (prec_mean * rec_mean) / (prec_mean + rec_mean)

print(f"Precision (COCO-based): {prec_mean:.3f}")
print(f"Recall    (COCO-based): {rec_mean:.3f}")
print(f"F1-Score  (COCO-based): {f1:.3f}\n")

### YOLOv12 Nano (Jetson-Simulated) Accuracy Metrics

mAP@50:95: 0.323
mAP@50:    0.469

Precision (COCO-based): 0.469
Recall    (COCO-based): 0.693
F1-Score  (COCO-based): 0.559

